In [59]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("s8a-dataframes-actividades").getOrCreate()

df = spark.read.option("header", "true").option("inferSchema", "true").csv("../VentasNulos.csv")
    
df.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Ventas: integer (nullable = true)
 |-- Euros: integer (nullable = true)
 |-- Ciudad: string (nullable = true)
 |-- Identificador: string (nullable = true)



In [60]:
df = df.na.drop("all")
df.show()

+------+------+-----+-----------+-------------+
|Nombre|Ventas|Euros|     Ciudad|Identificador|
+------+------+-----+-----------+-------------+
|  Pepe|     4|  200|      Elche|          X21|
|Andreu|     8| null|       null|         null|
|  Juan|  null| null|       null|          C54|
| Pedro|     1|   30|   Valencia|          R23|
| María|  null|  300| Torrellano|         null|
|Marina|     3|  350|       Aspe|          V55|
|  null|    10|  500|Crevillente|          AMV|
|   Ana|    10| 2300|   Alicante|          B89|
| Jorge|     8| null|       null|          T19|
+------+------+-----+-----------+-------------+



In [52]:
df = df.na.fill("Empleado",["Nombre"])
df.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| null|       null|         null|
|    Juan|  null| null|       null|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|  null|  300| Torrellano|         null|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|  null| null|       null|         null|
|   Jorge|     8| null|       null|          T19|
+--------+------+-----+-----------+-------------+



In [53]:
valor = df.groupBy().avg('Ventas')
media = valor.first()[0]

In [54]:
type(valor)

pyspark.sql.dataframe.DataFrame

In [55]:
# Las ventas nulas por la media de las ventas de los compañeros(redondeado a entero)
from pyspark.sql.functions import avg, col
df = df.fillna({"Ventas" : media})
df.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8| null|       null|         null|
|    Juan|     6| null|       null|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         null|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6| null|       null|         null|
|   Jorge|     8| null|       null|          T19|
+--------+------+-----+-----------+-------------+



In [56]:
# Los euros nulos por el valor del compañero que menos € ha ganado
valor = df.groupBy().min('Euros')
menor = valor.first()[0]
menor

30

In [57]:
df = df.fillna({"Euros" : menor})
df.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       null|         null|
|    Juan|     6|   30|       null|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|         null|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       null|         null|
|   Jorge|     8|   30|       null|          T19|
+--------+------+-----+-----------+-------------+



In [58]:
df = df.fillna({"Ciudad": "C.V.", "Identificador": "XYZ"})
df.show()

+--------+------+-----+-----------+-------------+
|  Nombre|Ventas|Euros|     Ciudad|Identificador|
+--------+------+-----+-----------+-------------+
|    Pepe|     4|  200|      Elche|          X21|
|  Andreu|     8|   30|       C.V.|          XYZ|
|    Juan|     6|   30|       C.V.|          C54|
|   Pedro|     1|   30|   Valencia|          R23|
|   María|     6|  300| Torrellano|          XYZ|
|  Marina|     3|  350|       Aspe|          V55|
|Empleado|    10|  500|Crevillente|          AMV|
|     Ana|    10| 2300|   Alicante|          B89|
|Empleado|     6|   30|       C.V.|          XYZ|
|   Jorge|     8|   30|       C.V.|          T19|
+--------+------+-----+-----------+-------------+

